# Astro Vibe Bot — Local Test Notebook

Use this notebook to generate a daily context, preview the channel message for all 12 signs, and optionally send it to a Telegram chat or channel.

Prereqs:
- `.env` must exist with `TELEGRAM_BOT_TOKEN` and `OPENAI_API_KEY`
- Optional: set `RSS_FEED_URL`, `TIMEZONE`, and `TEST_CHAT_ID` in `.env`


In [9]:
import os
import asyncio
from pathlib import Path
from zoneinfo import ZoneInfo

from dotenv import load_dotenv
from openai import OpenAI
from aiogram import Bot
import json

from main import load_signs, generate_daily_context, build_channel_sign_messages

root_dir = Path.cwd()
if not (root_dir / ".env").exists() and (root_dir.parent / ".env").exists():
    root_dir = root_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RSS_FEED_URL = os.getenv("RSS_FEED_URL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TEST_CHAT_ID = os.getenv("TEST_CHAT_ID")
TIMEZONE = os.getenv("TIMEZONE", "UTC")
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
TELEGRAM_NEWS_CHANNEL = os.getenv("TELEGRAM_NEWS_CHANNEL")
TELEGRAM_NEWS_LIMIT = int(os.getenv("TELEGRAM_NEWS_LIMIT", "20"))
TELETHON_SESSION = os.getenv("TELETHON_SESSION", "telethon.session")
TELETHON_SESSION_STRING = os.getenv("TELETHON_SESSION_STRING")

if not TELEGRAM_BOT_TOKEN or not OPENAI_API_KEY:
    raise RuntimeError("Set TELEGRAM_BOT_TOKEN and OPENAI_API_KEY in .env")

client = OpenAI(api_key=OPENAI_API_KEY)
signs = load_signs()
timezone = ZoneInfo(TIMEZONE)
telegram_source = {
    "api_id": int(TELEGRAM_API_ID) if TELEGRAM_API_ID else None,
    "api_hash": TELEGRAM_API_HASH,
    "channel": TELEGRAM_NEWS_CHANNEL,
    "limit": TELEGRAM_NEWS_LIMIT,
    "session_path": TELETHON_SESSION,
    "session_string": TELETHON_SESSION_STRING,
}


In [ ]:
# Generate today's global context
context = await generate_daily_context(
    client,
    signs,
    RSS_FEED_URL,
    OPENAI_MODEL,
    telegram_source=telegram_source,
)
context["global_summary"], list(context["vibes"].keys())[:3]


[debug] Telegram news context:
- ⚡️Сили оборони пошкодили технічну споруду обслуговування балістичних ракет середньої дальності на полігоні "Капустин Яр" Астраханської області рф Крім цього уражено: ▪️В районі Розівки – склад матеріально-технічного забезпечення. ▪️У районі Красногірського – зосередження живої сили противника. ▪️У Новоекономічному – пункт управління БпЛА ворога. НОВИНА | Підписатися
- ❗️Наразі ситуація не дозволяє скасувати аварійні відключення у більшості регіонів України, – Укренерго Водночас у частині областей обсяг застосованих обмежень сьогодні менший, ніж учора. Атомна генерація все ще частково розвантажена. Оприлюднені напередодні графіки погодинних знеструмлень у більшості регіонів наразі не діють. НОВИНА | Підписатися
- Україна запроваджує нові санкції проти компаній, які допомагають рф, – Зеленський Йдеться про: ▪️постачальників компонентів і виробників ракет та дронів; ▪️російський фінансовий сектор. Зокрема, компанії, через які оплачуються поставки компонент

('Ситуація в країні залишається напруженою, але спостерігається прогрес у боротьбі за стабільність.',
 ['Aries', 'Taurus', 'Gemini'])

In [3]:
# Build the daily channel messages (12 separate posts)
channel_messages = build_channel_sign_messages(context, signs)
channel_messages


['Сьогодні ви здатні подолати будь-які труднощі!\nСкладні часи потребують витривалості та підтримки одне одного.\n\n♈ Овен: Цей день вимагає від вас сміливості та рішучості. Ваша енергія зараз на висоті, використовуйте її для досягнення цілей. У стосунках проявіть свою відданість, адже ваша пристрасть зараз вражає. Грошові питання можуть бути нестабільними, але ви знайдете спосіб вирішити їх.',
 '♉ Телець: Сьогодні варто зосередитися на стабільності та спокої. Ваш практичний підхід допоможе вам впоратися з будь-якими викликами. У коханні ваша обережність принесе гармонію та спокій. У фінансах зберігайте обережність, не ризикуйте зайвий раз.',
 "♊ Близнюки: Сьогодні ви можете виявити свою допитливість у нових проектах. Ваші ідеї можуть стати основою для успіху. У стосунках будьте відкритими до спілкування, це зміцнить зв'язки. Фінансові можливості можуть з'явитися несподівано, тож будьте готові до змін.",
 '♋ Рак: Приділіть увагу своїм емоціям і потребам близьких сьогодні. Ваша чутливіс

In [4]:
# Optional: send the daily channel messages
# Set TEST_CHAT_ID in .env (example: your numeric chat_id or @TarotReaderOracle)

if TEST_CHAT_ID:
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    for message in channel_messages:
        await bot.send_message(TEST_CHAT_ID, message)
    await bot.session.close()
    "Sent"
else:
    "TEST_CHAT_ID not set"


In [ ]:
# Prompt sandbox: change prompt and run a single prediction
sign = "Taurus"
question = "Чи варто сьогодні ризикувати у фінансах?"

custom_system = "Ти лаконічний астрологічний радник. Відповідай лише українською."
custom_user = (
    f"Знак користувача: {sign}\n"
    f"Риси: {', '.join(signs[sign]['traits'])}\n"
    f"Специфіка: {signs[sign]['specificity']}\n"
    f"Вайб дня: {context.get('vibes', {}).get(sign, '')}\n"
    f"Глобальний підсумок: {context.get('global_summary', '')}\n\n"
    f"Питання користувача: {question}\n\n"
    "Відповідай у 3–5 реченнях, без категоричних тверджень."
)

response = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": custom_system},
        {"role": "user", "content": custom_user},
    ],
    temperature=0.7,
)

response.choices[0].message.content.strip()


In [ ]:
import os
from telethon import TelegramClient
from telethon.sessions import StringSession

api_id = int(os.environ["TELEGRAM_API_ID"])
api_hash = os.environ["TELEGRAM_API_HASH"]

async with TelegramClient(StringSession(), api_id, api_hash) as client:
    session_string = client.session.save()
session_string


In [12]:
# Generate today's global context
context = await generate_daily_context(
    client,
    signs,
    RSS_FEED_URL,
    OPENAI_MODEL,
    telegram_source=telegram_source,
)
context["global_summary"], list(context["vibes"].keys())[:3]


[debug] Telegram news context:
- ⚡️Сили оборони пошкодили технічну споруду обслуговування балістичних ракет середньої дальності на полігоні "Капустин Яр" Астраханської області рф Крім цього уражено: ▪️В районі Розівки – склад матеріально-технічного забезпечення. ▪️У районі Красногірського – зосередження живої сили противника. ▪️У Новоекономічному – пункт управління БпЛА ворога. НОВИНА | Підписатися
- ❗️Наразі ситуація не дозволяє скасувати аварійні відключення у більшості регіонів України, – Укренерго Водночас у частині областей обсяг застосованих обмежень сьогодні менший, ніж учора. Атомна генерація все ще частково розвантажена. Оприлюднені напередодні графіки погодинних знеструмлень у більшості регіонів наразі не діють. НОВИНА | Підписатися
- Україна запроваджує нові санкції проти компаній, які допомагають рф, – Зеленський Йдеться про: ▪️постачальників компонентів і виробників ракет та дронів; ▪️російський фінансовий сектор. Зокрема, компанії, через які оплачуються поставки компонент

('Ситуація в країні залишається складною, але є ознаки позитивних змін.',
 ['Aries', 'Taurus', 'Gemini'])